In [0]:
# !pip install tensorflow 
# TensorFlow and tf.keras
# its just working with tensorflow 1.13.1, with others has problem
import tensorflow as tf
print(tf.__version__)


1.14.0


**- goolge Colab Configuration**

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


**- Setting root address to project in Google Drive**

In [0]:
!ls "/content/gdrive/My Drive/NLPHW3"
root_path = '/content/gdrive/My Drive/NLPHW3'

 BackUpCodes
'Basic BDLSTM '
 BASIC_BDLSTM_Tagger.ipynb
 Data_Prepare_PKL_Format.ipynb
 Evaluation_Datasets
 LEX_BDLSTM_models_46.04966139954853
 LEX_BDLSTM_models_46.27539503386004
 LEX_BDLSTM_models_47.629796839729124
 LEX_BDLSTM_models_48.306997742663654
 LEX_BDLSTM_Tagger.ipynb
 Mask_LSTM_Model1.ipynb
 MFS_Tagger.ipynb
 POS_BDLSTM_models_20.118343195266274
 POS_BDLSTM_models_21.301775147928996
 POS_BDLSTM_models_23.668639053254438
 POS_BDLSTM_models_24.85207100591716
 POS_BDLSTM_models_25.443786982248522
 POS_BDLSTM_Tagger.ipynb
 POS_LEX_BDLSTM_Tagger.ipynb
 POS_LEX_WnDomain_BDLSTM_Tagger.ipynb
 POS_WnDomain_BDLSTM_Tagger.ipynb
 resources
 semcor
 semcor+omsti
 Test_Prediction_Write.ipynb
 Training_Corpora
 WnDomain_BDLSTM_models_46.72686230248307
 WnDomain_BDLSTM_models_47.17832957110609
 WnDomain_BDLSTM_models_48.53273137697517
 WnDomain_BDLSTM_models_48.75846501128668
 WnDomain_BDLSTM_models_50.112866817155755
 WnDomain_BDLSTM_Tagger.ipynb
 WSD_Evaluation_Framework


In [0]:
# Imports
from collections import Counter
import codecs
import xml.etree.cElementTree as etree
import pickle
import nltk
nltk.download('wordnet')
import numpy as np
import copy
from nltk.corpus import wordnet as wn


# Added Chars to dictionaries
UNK = "<UNK>"
PAD = "<PAD>"


RESOURCE_PATH = root_path+'/resources/'

MAP_WN2BN_FILE = RESOURCE_PATH + 'babelnet2wordnet.tsv'
MAP_BN2WNDOMAIN_FILE = RESOURCE_PATH + 'babelnet2wndomains.tsv'
MAP_BN2LEXNAMES_FILE = RESOURCE_PATH + 'babelnet2lexnames.tsv'


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
## -----------------------------------------------------------------------------
# function for getting dictionary of gold keys ->instance id: senseKey
def getGoldKey(goldKeyFile):
    print('getGoldKey is started ....')
    goldAddr = []
    goldKeys = []
    with codecs.open(goldKeyFile,'rb') as f:        
        for line in f:            
            line_ = line.decode().strip().split(' ')  
            goldAddr.append(line_[0])
            goldKeys.append(line_[1]) 
    goldKeysDic = dict(zip(goldAddr,goldKeys))
    
    print('getGoldKey is done ....')

    return goldKeysDic

# -----------------------------------------------------------------------------
def WN_to_BN_dic(mapping_file):
    print('WN_to_BN_dic is started ....')
    BN_ID = []
    WN_ID = []
    with codecs.open(mapping_file,'rb') as f:        
        for line in f:            
            line_synsets = line.decode().strip().split('\t')
            BN_ID.append(line_synsets[0])
            WN_ID.append(line_synsets[1])    
    WN2BN_map_dic = dict(zip(WN_ID,BN_ID))
    print('WN_to_BN_dic is done ....')

    return WN2BN_map_dic


# -----------------------------------------------------------------------------
def BN_to_WNDOMAIN_dic(mapping_file):
    print('WN_to_BN_dic is started ....')
    BN_ID = []
    WN_DOMAIN = []
    with codecs.open(mapping_file,'rb') as f:        
        for line in f:            
            line_synsets = line.decode().strip().split('\t')
            BN_ID.append(line_synsets[0])
            WN_DOMAIN.append(line_synsets[1])    
    BN2WnDomain_dic = dict(zip(BN_ID, WN_DOMAIN))
    print('BN_to_WNDOMAIN_dic is done ....')
    return BN2WnDomain_dic



# -----------------------------------------------------------------------------
def BN_to_LexNames_dic(mapping_file):
    print('BN_to_LexNames is started ....')
    BN_ID = []
    LEXNAMES = []
    with codecs.open(mapping_file,'rb') as f:        
        for line in f:            
            line_synsets = line.decode().strip().split('\t')
            BN_ID.append(line_synsets[0])
            LEXNAMES.append(line_synsets[1])    
    BN2LexNames_dic = dict(zip(BN_ID, LEXNAMES))
    print('BN_to_LexNames_dic is done ....')

    return BN2LexNames_dic



## ----------------------------------------------------------------------------
def getMFS_(word, WN2BN_map_dic):
    MFS_wnId = UNK
    all_synsets = wn.synsets(word)

    # check if the word has synsets or not!
    if len(all_synsets) == 0 or all_synsets is None:
        return word
    
    synset = all_synsets[0]
    MFS_wnId = "wn:" + str(synset.offset()).zfill( 8) + synset.pos()
    if WN2BN_map_dic.get(MFS_wnId) is not None:
        MFS_bnId = WN2BN_map_dic.get(MFS_wnId)
        
    return MFS_bnId

#mfs = getMFS_('how')


def getPOSTagOfCharPOS(pos):
    if pos == 'n':
        return 'noun'
    if pos == 'v':
        return 'verb'
    if pos == 'a':
        return 'adj'
    if pos == 'r':
        return 'adv'

def get_WN_POSTagOfCharPOS(pos):
    if pos == 'noun':
        return wn.NOUN
    if pos == 'verb':
        return wn.VERB
    if pos == 'adj':
        return wn.ADJ
    if pos == 'adv':
        return wn.ADV

## ----------------------------------------------------------------------    
# function for getting all possible synsets of one lemma with considering the POS of lemma from WordNets in BabelNet id format
def get_bnSenseCandydtsAndPOSOfLemma(lemma, pos_lemma, WN2BN_map_dic):
    synsets_ = []
    pos_ = []
    for synset in wn.synsets(lemma, pos = get_WN_POSTagOfCharPOS(pos_lemma)): # pos=wn.VERB
        synset_id = "wn:" + str(synset.offset()).zfill( 8) + synset.pos() # out: wn:05899087n                
        if WN2BN_map_dic.get(synset_id) is not None:
            bnId_ = WN2BN_map_dic.get(synset_id)
#            print(WN2BN_map_dic.get(synset_id))
            synsets_.append(bnId_)
            pos_.append(getPOSTagOfCharPOS(bnId_[-1:]))

    return synsets_, pos_

#synsets_ = get_bnSenseCandydtsOfLemma('long' ,)


## ----------------------------------------------------------------------    
# get LexName of bnId
def get_LexOfLemmaSense(bnIds, BN2LexNames_dic):
    lexs_ = []
    for bnId_ in bnIds:
        if BN2LexNames_dic.get(bnId_) is not None:
            lexs_.append(BN2LexNames_dic.get(bnId_))
        else:
            lexs_.append('other')

    return lexs_

#lexs_ = get_LexOfLemmaSense(synsets_)
    


## ----------------------------------------------------------------------    
# get WnDomain of bnId
def get_WnDomainsOfLemmaSense(bnIds, BN2WnDomain_dic):
    wnDomains_ = []
    for bnId_ in bnIds:
        if BN2WnDomain_dic.get(bnId_) is not None:
            wnDomains_.append(BN2WnDomain_dic.get(bnId_))
        else:
            wnDomains_.append('other')

    return wnDomains_

#wnDomains_ = get_WnDomainsOfLemmaSense(synsets_)


# this is code for parsing xml file for getting all all data like instance id, pos, Sense, All possible Sense and ....
def parse_xml(filename, goldKeysDic, WN2BN_map_dic, BN2LexNames_dic, BN2WnDomain_dic):  
    print('parse_xml is started ....')   
    
    x_IdInstance_Sent, x_Raw_Sent, y_POS_Sent = [],[],[]
    y_BNsense_Sent, y_Lex_Sent, y_WnDomain_Sent, y_MFS_Sent = [],[],[],[]
    y_allSenses_Sent, y_allPos_Sent, y_allLexs_Sent, y_allwnDomains_Sent = [],[],[],[]
    
    IdInstance_sent, Raw_sent, POS_sent, BN_sent, Lex_sent, WnDomain_sent, MFS_sent,\
                    allSenses_sent, allpos_sent, allLexs_sent, allwnDomains_sent = [],[],[],[],[],[],[],[],[],[],[]
    
    context = etree.iterparse(filename, events=("start", "end"))   
    
    id_sent = 0;

    for event, element in context:            
        if event == "start":        
            if element.tag =='sentence':                
#                 if id_sent == 10:
#                     break
#                sentfile.write((str(id_sent)+'<M.J>').lower().encode())
                if id_sent % 5000 == 0:
                    print(id_sent)
                id_sent = id_sent + 1
                
            if element.tag == "wf":  
                id_ = 'None'
                lemma_ = (str(element.attrib['lemma'])).lower()
                pos_ = (str(element.attrib['pos'])).lower()                
                
                IdInstance_sent.append(id_)
                Raw_sent.append(lemma_)            
                POS_sent.append(pos_)
                BN_sent.append(lemma_)
                Lex_sent.append('other')
                WnDomain_sent.append('other')            
                MFS_sent.append(getMFS_(lemma_, WN2BN_map_dic))
                allSenses_sent.append([lemma_])
                allpos_sent.append([pos_])
                allLexs_sent.append(['other'])
                allwnDomains_sent.append(['other'])
                
            if element.tag == "instance": 
                instanceId_ = (str(element.attrib['id'])).lower()
                lemma_ = (str(element.attrib['lemma'])).lower()
                pos_ = (str(element.attrib['pos'])).lower()
                    
                IdInstance_sent.append(instanceId_)
                Raw_sent.append(lemma_)            
                POS_sent.append(pos_)
                
                SenseKey_ = goldKeysDic.get(instanceId_) # sense for specific instance from gold file
#                 print(SenseKey_)
                
                synset = wn.lemma_from_key(SenseKey_).synset() # out: Synset('long.a.01')
                synset_id = "wn:" + str(synset.offset()).zfill( 8) + synset.pos() # out: wn:05899087n                
                
                bnId_ = UNK
                if WN2BN_map_dic.get(synset_id) is not None:
                    bnId_ = WN2BN_map_dic.get(synset_id)                    
                else:
                    print("wnId:  ",synset_id, " is not in wn2bnMapping file.")
                
                BN_sent.append(bnId_)
            
                if BN2LexNames_dic.get(bnId_) is not None:
                    Lex_sent.append(BN2LexNames_dic.get(bnId_))
                else:
                    Lex_sent.append('other')
#                    print("bnId:  ", bnId_ , " is not in bn2lexMapping file.")

                if BN2WnDomain_dic.get(bnId_) is not None:
                    WnDomain_sent.append(BN2WnDomain_dic.get(bnId_))                    
                else:
                    WnDomain_sent.append('other')                    
#                    print("bnId:  ", bnId_ , " is not in bn2wnDomainMapping file.")
                                
                candSense, CandPos = get_bnSenseCandydtsAndPOSOfLemma(lemma_, pos_, WN2BN_map_dic)
                            
                MFS_sent.append(getMFS_(lemma_, WN2BN_map_dic))
                allSenses_sent.append(candSense)
                allpos_sent.append(CandPos)
                allLexs_sent.append(get_LexOfLemmaSense(candSense, BN2LexNames_dic))
                allwnDomains_sent.append(get_WnDomainsOfLemmaSense(candSense, BN2WnDomain_dic))
                                                                                    
        if event == "end":
            if element.tag == "sentence" :
                x_IdInstance_Sent.append(IdInstance_sent)
                x_Raw_Sent.append(Raw_sent)            
                y_POS_Sent.append(POS_sent)
                y_BNsense_Sent.append(BN_sent)
                y_Lex_Sent.append(Lex_sent)
                y_WnDomain_Sent.append(WnDomain_sent)
                y_MFS_Sent.append(MFS_sent)
                y_allSenses_Sent.append(allSenses_sent)
                y_allPos_Sent.append(allpos_sent)
                y_allLexs_Sent.append(allLexs_sent)
                y_allwnDomains_Sent.append(allwnDomains_sent) 
                
                IdInstance_sent, Raw_sent, POS_sent, BN_sent, Lex_sent, WnDomain_sent, MFS_sent,\
                    allSenses_sent, allpos_sent, allLexs_sent, allwnDomains_sent = [],[],[],[],[],[],[],[],[],[],[]
                
        element.clear()
        
    return x_IdInstance_Sent, x_Raw_Sent, y_POS_Sent,\
             y_BNsense_Sent, y_Lex_Sent, y_WnDomain_Sent, y_MFS_Sent,\
                 y_allSenses_Sent, y_allPos_Sent, y_allLexs_Sent, y_allwnDomains_Sent


# function for writing files as pkl file in disk
def get_sent_PKL_file(DATASET_FILE, save_path, goldKeysDic, WN2BN_map_dic, BN2LexNames_dic, BN2WnDomain_dic):                    
    # function for parsing large xml file and getting needed data from that.

    x_IdInstance_Sent, x_Raw_Sent, y_POS_Sent, y_BNsense_Sent, y_Lex_Sent, y_WnDomain_Sent,\
        y_MFS_Sent, y_allSenses_Sent, y_allPos_Sent, y_allLexs_Sent, y_allwnDomains_Sent\
    = parse_xml(DATASET_FILE, goldKeysDic, WN2BN_map_dic, BN2LexNames_dic, BN2WnDomain_dic)
    
    print('parse xml is ok .... ')
    
    with open(save_path + 'x_IdInstance_Sent.pkl', "wb") as outfile:
        pickle.dump(x_IdInstance_Sent, outfile)
        
    with open(save_path + 'x_Raw_Sent.pkl', "wb") as outfile:
        pickle.dump(x_Raw_Sent, outfile)
        
    with open(save_path + 'y_POS_Sent.pkl', "wb") as outfile:
        pickle.dump(y_POS_Sent, outfile)
        
    with open(save_path + 'y_BNsense_Sent.pkl', "wb") as outfile:
        pickle.dump(y_BNsense_Sent, outfile)
        
    with open(save_path + 'y_Lex_Sent.pkl', "wb") as outfile:
        pickle.dump(y_Lex_Sent, outfile)
        
    with open(save_path + 'y_WnDomain_Sent.pkl', "wb") as outfile:
        pickle.dump(y_WnDomain_Sent, outfile)
        
    with open(save_path + 'y_MFS_Sent.pkl', "wb") as outfile:
        pickle.dump(y_MFS_Sent, outfile)
        
    with open(save_path + 'y_allSenses_Sent.pkl', "wb") as outfile:
        pickle.dump(y_allSenses_Sent, outfile)
        
    with open(save_path + 'y_allPos_Sent.pkl', "wb") as outfile:
        pickle.dump(y_allPos_Sent, outfile)
        
    with open(save_path + 'y_allLex_Sent.pkl', "wb") as outfile:
        pickle.dump(y_allLexs_Sent, outfile)
        
    with open(save_path + 'y_allWnDomain_Sent.pkl', "wb") as outfile:
        pickle.dump(y_allwnDomains_Sent, outfile)
                
        
    print('PKL file saved successfully ......')
    


WN2BN_map_dic = WN_to_BN_dic(MAP_WN2BN_FILE)
print(len(WN2BN_map_dic))

BN2WnDomain_dic = BN_to_WNDOMAIN_dic(MAP_BN2WNDOMAIN_FILE)
print(len(BN2WnDomain_dic))

BN2LexNames_dic = BN_to_LexNames_dic(MAP_BN2LEXNAMES_FILE)
print(len(BN2LexNames_dic))


# DATASET_FILE = root_path + '/Training_Corpora/SemCor/semcor.data.xml'
# GOLD_KEY_FILE = root_path + '/Training_Corpora/SemCor/semcor.gold.key.txt'  
# PATH_TRAIN = root_path + '/semcor/semcor/'
# goldKeysDic = getGoldKey(GOLD_KEY_FILE)

# get_sent_PKL_file(DATASET_FILE, PATH_TRAIN, goldKeysDic, WN2BN_map_dic, BN2LexNames_dic, BN2WnDomain_dic)

file_names = ['semeval2007', 'semeval2013', 'semeval2015', 'senseval2', 'senseval3', 'ALL']


for file_name in file_names:
    DATASET_FILE = root_path + '/Evaluation_Datasets/{}/{}.data.xml'.format(file_name, file_name)
    GOLD_KEY_FILE = root_path + '/Evaluation_Datasets/{}/{}.gold.key.txt'.format(file_name, file_name) 
    PATH = root_path + '/semcor+omsti/{}/'.format(file_name)

    goldKeysDic = getGoldKey(GOLD_KEY_FILE)

    get_sent_PKL_file(DATASET_FILE, PATH, goldKeysDic, WN2BN_map_dic, BN2LexNames_dic, BN2WnDomain_dic)



WN_to_BN_dic is started ....
WN_to_BN_dic is done ....
117659
WN_to_BN_dic is started ....
BN_to_WNDOMAIN_dic is done ....
92601
BN_to_LexNames is started ....
BN_to_LexNames_dic is done ....
117653
getGoldKey is started ....
getGoldKey is done ....
parse_xml is started ....
0
parse xml is ok .... 
PKL file saved successfully ......
getGoldKey is started ....
getGoldKey is done ....
parse_xml is started ....
0
parse xml is ok .... 
PKL file saved successfully ......
getGoldKey is started ....
getGoldKey is done ....
parse_xml is started ....
0
parse xml is ok .... 
PKL file saved successfully ......
getGoldKey is started ....
getGoldKey is done ....
parse_xml is started ....
0
parse xml is ok .... 
PKL file saved successfully ......
getGoldKey is started ....
getGoldKey is done ....
parse_xml is started ....
0
parse xml is ok .... 
PKL file saved successfully ......
getGoldKey is started ....
getGoldKey is done ....
parse_xml is started ....
0
parse xml is ok .... 
PKL file saved succe

In [0]:
# this 5 function is for creating dictionaries of lemma words, Senses, POS, LexNames and WnDomains
## -----------------------------------------------------------------------------
def get_RawWords2id_Dic(x_Raw_Sent_pkl):
    print('get_RawWords2id_Dic is started ....')
        
    x_Raw_Sent = pickle.load(open(x_Raw_Sent_pkl, "rb"))

    RawWords2id = { PAD : 0, UNK: 1} 
    
    raw_words = []
    # get words from dataset
    for i in range(len(x_Raw_Sent)):
        for j in range(len(x_Raw_Sent[i])):
            raw_words.append(x_Raw_Sent[i][j])                        
                
    counter = Counter(raw_words)    
    count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))
    words, _ = list(zip(*count_pairs))
    RawWords2id.update(dict(zip(words, range(2, len(words) + 2))))

    print('get_RawWords2id is done ....')    
    return RawWords2id


## -----------------------------------------------------------------------------
def get_POS2id_Dic(y_POS_Sent_pkl, y_allPos_Sent_pkl):
    print('get_POS2id_Dic is started ....')
    
    y_POS_Sent = pickle.load(open(y_POS_Sent_pkl, "rb"))
    y_allPos_Sent = pickle.load(open(y_allPos_Sent_pkl, "rb"))    
    
    pos2id = { 'verb' : 0, 'noun': 1, 'adj' : 2, 'adv': 3}
    pos = []
    # get words from dataset
    for i in range(len(y_POS_Sent)):
        for j in range(len(y_POS_Sent[i])):                        
            pos.append(y_POS_Sent[i][j]) 
            
            for k in range(len(y_allPos_Sent[i][j])):
                pos.append(y_allPos_Sent[i][j][k])                            
                  
        
    counter = Counter(pos)    
    count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))
    words, _ = list(zip(*count_pairs))
    
    pos_words = []
    for p in range(len(words)):
        if words[p] != 'verb':
            if words[p] != 'noun':
                if words[p] != 'adj':
                    if words[p] != 'adv':
                        pos_words.append(words[p])    
    
    pos2id.update(dict(zip(pos_words, range(4, len(pos_words) + 4))))


    print('get_POS2id_Dic is done ....')    
    return pos2id



## -----------------------------------------------------------------------------
def get_Senses2id_Dic(y_allSenses_Sent_pkl, y_BNsense_Sent_pkl):
    print('get_Senses2id_Dic is started ....')
    
    y_allSenses_Sent = pickle.load(open(y_allSenses_Sent_pkl, "rb"))
    y_BNsense_Sent = pickle.load(open(y_BNsense_Sent_pkl, "rb"))
    
    SenseWords2id = {UNK: 0} 
    sense_words = []
    # get words from dataset
    for i in range(len(y_allSenses_Sent)):
        for j in range(len(y_allSenses_Sent[i])):
            sense_words.append(y_BNsense_Sent[i][j])
            
            for k in range(len(y_allSenses_Sent[i][j])):
                sense_words.append(y_allSenses_Sent[i][j][k])
                                
    
    counter = Counter(sense_words)    
    count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))
    all_senseANDwords, _ = list(zip(*count_pairs))
    SenseWords2id.update(dict(zip(all_senseANDwords, range(1, len(all_senseANDwords) + 1))))                

    print('get_Senses2id_Dic is done ....')    
    return SenseWords2id


## -----------------------------------------------------------------------------
def get_LEX2id_Dic(y_allLexs_Sent_pkl):
    print('get_LEX2id_Dic is started ....')
    
    y_allLexs_Sent = pickle.load(open(y_allLexs_Sent_pkl, "rb"))    
    
    lex = []
    # get words from dataset
    for i in range(len(y_allLexs_Sent)):
        for j in range(len(y_allLexs_Sent[i])):            
            for k in range(len(y_allLexs_Sent[i][j])):
                lex.append(y_allLexs_Sent[i][j][k])                                        
    
    counter = Counter(lex)    
    count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))
    words, _ = list(zip(*count_pairs))
    lex2id = dict(zip(words, range(len(words))))        

    print('get_2idDictionaries is done ....')    
    return lex2id



## -----------------------------------------------------------------------------
def get_2WnDomain2id_Dic(y_allwnDomains_Sent_pkl):
    print('get_2WnDomain2id_Dic is started ....')
    
    y_allwnDomains_Sent = pickle.load(open(y_allwnDomains_Sent_pkl, "rb")) 
    
    wnDomain = []
    # get words from dataset
    for i in range(len(y_allwnDomains_Sent)):
        for j in range(len(y_allwnDomains_Sent[i])):            
            for k in range(len(y_allwnDomains_Sent[i][j])):
                wnDomain.append(y_allwnDomains_Sent[i][j][k])
                    
    counter = Counter(wnDomain)    
    count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))
    words, _ = list(zip(*count_pairs))
    wnDomain2id = dict(zip(words, range(len(words))))

    print('get_2idDictionaries is done ....')    
    return wnDomain2id



In [0]:
dataset_file = 'semcor'

RawWords2id = get_RawWords2id_Dic(root_path + '/{}/{}/x_Raw_Sent.pkl'.format(dataset_file,dataset_file))

pos2id = get_POS2id_Dic(root_path + '/{}/{}/y_POS_Sent.pkl'.format(dataset_file,dataset_file), root_path + '/{}/{}/y_allPos_Sent.pkl'.format(dataset_file,dataset_file))

SenseWords2id = get_Senses2id_Dic(root_path + '/{}/{}/y_allSenses_Sent.pkl'.format(dataset_file,dataset_file), root_path + '/{}/{}/y_BNsense_Sent.pkl'.format(dataset_file,dataset_file))

lex2id = get_LEX2id_Dic(root_path + '/{}/{}/y_allLex_Sent.pkl'.format(dataset_file,dataset_file))

wnDomain2id = get_2WnDomain2id_Dic(root_path + '/{}/{}/y_allWnDomain_Sent.pkl'.format(dataset_file,dataset_file))


all_dict_data = {}    
all_dict_data["RawWords2id"] = RawWords2id
all_dict_data["SenseWords2id"] = SenseWords2id
all_dict_data["pos2id"] = pos2id
all_dict_data["lex2id"] = lex2id
all_dict_data["wnDomain2id"] = wnDomain2id

with open(root_path + '/{}/{}/all_dict_data.pkl'.format(dataset_file,dataset_file), "wb") as outfile:
    pickle.dump(all_dict_data, outfile)

get_RawWords2id_Dic is started ....
get_RawWords2id is done ....
get_POS2id_Dic is started ....
get_POS2id_Dic is done ....
get_Senses2id_Dic is started ....
get_Senses2id_Dic is done ....
get_LEX2id_Dic is started ....
get_2idDictionaries is done ....
get_2WnDomain2id_Dic is started ....
get_2idDictionaries is done ....


In [0]:
dataset_file = 'semcor'

all_dict_data = pickle.load(open(root_path+'/{}/{}/all_dict_data.pkl'.format(dataset_file,dataset_file), "rb"))

RawWords2id = all_dict_data["RawWords2id"]
SenseWords2id = all_dict_data["SenseWords2id"] 
pos2id = all_dict_data["pos2id"] 
lex2id = all_dict_data["lex2id"] 
wnDomain2id = all_dict_data["wnDomain2id"]

print(len(pos2id))
print(len(SenseWords2id))

12
60919


In [0]:
# this 5 functions are for converting data to id
## -----------------------------------------------------------------------------
def get_Raw_data_2id(x_Raw_Sent_pkl, RawWords2id, dataset_name, file_name):   
    print('get_Raw_data_2id is started ....')
    
    x_Raw_Sent = pickle.load(open(x_Raw_Sent_pkl, "rb")) 
    x_Raw_Sent_id = [[] for i in range(len(x_Raw_Sent))]        
    
    for i in range(len(x_Raw_Sent)):
        for j in range(len(x_Raw_Sent[i])):                        
            x_Raw_Sent_id[i].append(RawWords2id.get(x_Raw_Sent[i][j], RawWords2id[UNK]))            
            
    with open(root_path + '/{}/{}/x_Raw_Sent_id.pkl'.format(dataset_name, file_name), "wb") as outfile:
        pickle.dump(x_Raw_Sent_id, outfile)
        
    print('get_Raw_data_2id is done ....')                
#     return x_Raw_Sent_id



## -----------------------------------------------------------------------------
def get_BNSense_data_2id(y_BNsense_Sent_pkl, y_allSenses_Sent_pkl, SenseWords2id, dataset_name, file_name):   
    print('get_BNSense_data_2id is started ....')

    y_BNsense_Sent = pickle.load(open(y_BNsense_Sent_pkl, "rb")) 
    y_allSenses_Sent = pickle.load(open(y_allSenses_Sent_pkl, "rb")) 

    y_BNsense_Sent_id = [[] for i in range(len(y_BNsense_Sent))]    
    y_allSenses_Sent_id = [[] for i in range(len(y_allSenses_Sent))]    
    
    for i in range(len(y_BNsense_Sent)):
        for j in range(len(y_BNsense_Sent[i])):                                             
            y_BNsense_Sent_id[i].append(SenseWords2id.get(y_BNsense_Sent[i][j], SenseWords2id[UNK]))  
                                      
            y_allSenses_Sent_id[i].append([])
                        
            for k in range(len(y_allSenses_Sent[i][j])):                                            
                y_allSenses_Sent_id[i][j].append(SenseWords2id.get(y_allSenses_Sent[i][j][k], SenseWords2id[UNK]))  
                
    with open(root_path + '/{}/{}/y_BNsense_Sent_id.pkl'.format(dataset_name, file_name), "wb") as outfile:
        pickle.dump(y_BNsense_Sent_id, outfile)
        
    with open(root_path + '/{}/{}/y_allSenses_Sent_id.pkl'.format(dataset_name, file_name), "wb") as outfile:
        pickle.dump(y_allSenses_Sent_id, outfile)
        
    print('get_BNSense_data_2id is done ....')                
#     return y_BNsense_Sent_id, y_allSenses_Sent_id


## -----------------------------------------------------------------------------
def get_POS_data_2id(y_POS_Sent_pkl, y_allPos_Sent_pkl, pos2id, dataset_name, file_name):   
    print('get_POS_data_2id is started ....')
        
    y_POS_Sent = pickle.load(open(y_POS_Sent_pkl, "rb")) 
    y_allPos_Sent = pickle.load(open(y_allPos_Sent_pkl, "rb"))
    
    y_POS_Sent_id = [[] for i in range(len(y_POS_Sent))]     
    y_allPos_Sent_id = [[] for i in range(len(y_allPos_Sent))]        
    
    for i in range(len(y_POS_Sent)):
        for j in range(len(y_POS_Sent[i])):                                    
            y_POS_Sent_id[i].append(pos2id.get(y_POS_Sent[i][j]))   
            
            y_allPos_Sent_id[i].append([])           
            
            for k in range(len(y_allPos_Sent[i][j])):                                                            
                y_allPos_Sent_id[i][j].append(pos2id.get(y_allPos_Sent[i][j][k]))                

    with open(root_path + '/{}/{}/y_POS_Sent_id.pkl'.format(dataset_name, file_name), "wb") as outfile:
        pickle.dump(y_POS_Sent_id, outfile)
        
    with open(root_path + '/{}/{}/y_allPos_Sent_id.pkl'.format(dataset_name, file_name), "wb") as outfile:
        pickle.dump(y_allPos_Sent_id, outfile)
        
    print('get_POS_data_2id is done ....')                
#     return y_POS_Sent_id, y_allPos_Sent_id



## -----------------------------------------------------------------------------
def get_LEX_data_2id(y_Lex_Sent_pkl, y_allLexs_Sent_pkl, lex2id, dataset_name, file_name):   
    print('get_LEX_data_2id is started ....')

    y_Lex_Sent = pickle.load(open(y_Lex_Sent_pkl, "rb")) 
    y_allLexs_Sent = pickle.load(open(y_allLexs_Sent_pkl, "rb"))
    
    y_Lex_Sent_id = [[] for i in range(len(y_Lex_Sent))]
    y_allLexs_Sent_id = [[] for i in range(len(y_allLexs_Sent))]
           
    for i in range(len(y_Lex_Sent)):
        for j in range(len(y_Lex_Sent[i])):                                    
            y_Lex_Sent_id[i].append(lex2id.get(y_Lex_Sent[i][j]))                                   
            
            y_allLexs_Sent_id[i].append([])
            
            for k in range(len(y_allLexs_Sent[i][j])):                                                            
                y_allLexs_Sent_id[i][j].append(lex2id.get(y_allLexs_Sent[i][j][k]))                
    
    with open(root_path + '/{}/{}/y_Lex_Sent_id.pkl'.format(dataset_name, file_name), "wb") as outfile:
        pickle.dump(y_Lex_Sent_id, outfile)
        
    with open(root_path + '/{}/{}/y_allLex_Sent_id.pkl'.format(dataset_name, file_name), "wb") as outfile:
        pickle.dump(y_allLexs_Sent_id, outfile)
        
    print('get_LEX_data_2id is done ....')                
#     return y_Lex_Sent_id, y_allLexs_Sent_id


## -----------------------------------------------------------------------------
def get_WnDomain_data_2id(y_WnDomain_Sent_pkl, y_allwnDomains_Sent_pkl, wnDomain2id, dataset_name, file_name):   
    print('get_WnDomain_data_2id is started ....')
        
    y_WnDomain_Sent = pickle.load(open(y_WnDomain_Sent_pkl, "rb")) 
    y_allwnDomains_Sent = pickle.load(open(y_allwnDomains_Sent_pkl, "rb"))
    
    y_WnDomain_Sent_id = [[] for i in range(len(y_WnDomain_Sent))]               
    y_allwnDomains_Sent_id = [[] for i in range(len(y_allwnDomains_Sent))]
    
    
    for i in range(len(y_WnDomain_Sent)):
        for j in range(len(y_WnDomain_Sent[i])):                                    
            y_WnDomain_Sent_id[i].append(wnDomain2id.get(y_WnDomain_Sent[i][j]))              
            y_allwnDomains_Sent_id[i].append([])
            
            for k in range(len(y_allwnDomains_Sent[i][j])):                                                            
                y_allwnDomains_Sent_id[i][j].append(wnDomain2id.get(y_allwnDomains_Sent[i][j][k])) 
    
    with open(root_path + '/{}/{}/y_WnDomain_Sent_id.pkl'.format(dataset_name, file_name), "wb") as outfile:
        pickle.dump(y_WnDomain_Sent_id, outfile)
        
    with open(root_path + '/{}/{}/y_allWnDomain_Sent_id.pkl'.format(dataset_name, file_name), "wb") as outfile:
        pickle.dump(y_allwnDomains_Sent_id, outfile)
        
    print('get_WnDomain_data_2id is done ....')                
#     return y_WnDomain_Sent_id, y_allwnDomains_Sent_id


In [0]:
file_names = ['semcor', 'ALL', 'senseval2', 'senseval3', 'semeval2007', 'semeval2013', 'semeval2015']
dataset_name = 'semcor'
for file_name in file_names:

    get_Raw_data_2id(root_path + '/{}/{}/x_Raw_Sent.pkl'.format(dataset_name, file_name), RawWords2id,dataset_name, file_name)

    get_BNSense_data_2id(root_path + '/{}/{}/y_BNsense_Sent.pkl'.format(dataset_name, file_name), root_path + '/{}/{}/y_allSenses_Sent.pkl'.format(dataset_name, file_name), SenseWords2id, dataset_name, file_name)

    get_POS_data_2id(root_path + '/{}/{}/y_POS_Sent.pkl'.format(dataset_name, file_name), root_path + '/{}/{}/y_allPos_Sent.pkl'.format(dataset_name, file_name), pos2id, dataset_name, file_name)

    get_LEX_data_2id(root_path + '/{}/{}/y_Lex_Sent.pkl'.format(dataset_name, file_name), root_path + '/{}/{}/y_allLex_Sent.pkl'.format(dataset_name, file_name), lex2id, dataset_name, file_name)

    get_WnDomain_data_2id(root_path + '/{}/{}/y_WnDomain_Sent.pkl'.format(dataset_name, file_name), root_path + '/{}/{}/y_allWnDomain_Sent.pkl'.format(dataset_name, file_name), wnDomain2id, dataset_name, file_name)
    
    print('processing is ok for -----> ', file_name)

get_Raw_data_2id is started ....
get_Raw_data_2id is done ....
get_BNSense_data_2id is started ....
get_BNSense_data_2id is done ....
get_POS_data_2id is started ....
get_POS_data_2id is done ....
get_LEX_data_2id is started ....
get_LEX_data_2id is done ....
get_WnDomain_data_2id is started ....
get_WnDomain_data_2id is done ....
processing is ok for ----->  semcor
get_Raw_data_2id is started ....
get_Raw_data_2id is done ....
get_BNSense_data_2id is started ....
get_BNSense_data_2id is done ....
get_POS_data_2id is started ....
get_POS_data_2id is done ....
get_LEX_data_2id is started ....
get_LEX_data_2id is done ....
get_WnDomain_data_2id is started ....
get_WnDomain_data_2id is done ....
processing is ok for ----->  ALL
get_Raw_data_2id is started ....
get_Raw_data_2id is done ....
get_BNSense_data_2id is started ....
get_BNSense_data_2id is done ....
get_POS_data_2id is started ....
get_POS_data_2id is done ....
get_LEX_data_2id is started ....
get_LEX_data_2id is done ....
get_W

In [0]:
# this 3 function is for creating masks of pos, lex and WnDomain to sense vocabulary
# our method is creatin an 2D list with size of pos dic * size of sense dic, and then put 1 in places that [pos][sense], we do the same for othe masks too.

## ----------------------------------------------------------------------------
def mask_PosToSenses(y_allPos_Sent_pkl, y_allSenses_Sent_id_pkl, pos2id, SenseWords2id, dataset_name, file_name):
    print('mask_PosToSenses is started ....')
    
    y_allPos_Sent = pickle.load(open(y_allPos_Sent_pkl, "rb"))
    y_allSenses_Sent_id = pickle.load(open(y_allSenses_Sent_id_pkl, "rb"))
    
    mask_pos2sense = [0]*np.ones(shape = (len(pos2id), len(SenseWords2id)))
    
    for i in range(len(y_allPos_Sent)):
        for j in range(len(y_allPos_Sent[i])):
            for k in range(len(y_allPos_Sent[i][j])):
                if pos2id.get(y_allPos_Sent[i][j][k]) is not None:
                    mask_pos2sense[pos2id[y_allPos_Sent[i][j][k]]][y_allSenses_Sent_id[i][j][k]] = 1                                    
    
    with open(root_path + '/{}/{}/mask_pos2sense.pkl'.format(dataset_name, file_name), "wb") as outfile:
        pickle.dump(mask_pos2sense, outfile)
        
    print('mask_PosToSenses is done ....')
#     return mask_pos2sense   


## ----------------------------------------------------------------------------
def mask_LexToSenses(y_allLexs_Sent_pkl, y_allSenses_Sent_id_pkl, lex2id, SenseWords2id, dataset_name, file_name):
    print('mask_LexToSenses is started ....')
    
    y_allLexs_Sent = pickle.load(open(y_allLexs_Sent_pkl, "rb"))
    y_allSenses_Sent_id = pickle.load(open(y_allSenses_Sent_id_pkl, "rb"))
    
    mask_lex2sense = [0]*np.ones(shape = (len(lex2id), len(SenseWords2id)))
    
    for i in range(len(y_allLexs_Sent)):
        for j in range(len(y_allLexs_Sent[i])):
            for k in range(len(y_allLexs_Sent[i][j])):
                if lex2id.get(y_allLexs_Sent[i][j][k]) is not None:
                    mask_lex2sense[lex2id[y_allLexs_Sent[i][j][k]]][y_allSenses_Sent_id[i][j][k]] = 1    
    
    with open(root_path + '/{}/{}/mask_lex2sense.pkl'.format(dataset_name, file_name), "wb") as outfile:
        pickle.dump(mask_lex2sense, outfile)
        
    print('mask_LexToSenses is done ....')                                
#     return mask_lex2sense   



## ----------------------------------------------------------------------------
def mask_WnDomansToSenses(y_allwnDomains_Sent_pkl, y_allSenses_Sent_id_pkl, wnDomain2id, SenseWords2id, dataset_name, file_name):
    print('mask_WnDomansToSenses is started ....')
    
    y_allwnDomains_Sent = pickle.load(open(y_allwnDomains_Sent_pkl, "rb"))
    y_allSenses_Sent_id = pickle.load(open(y_allSenses_Sent_id_pkl, "rb"))    
    
    mask_wnDomain2sense = [0]*np.ones(shape = (len(wnDomain2id), len(SenseWords2id)))
    
    for i in range(len(y_allwnDomains_Sent)):
        for j in range(len(y_allwnDomains_Sent[i])):
            for k in range(len(y_allwnDomains_Sent[i][j])):
                if wnDomain2id.get(y_allwnDomains_Sent[i][j][k]) is not None:
                    mask_wnDomain2sense[wnDomain2id[y_allwnDomains_Sent[i][j][k]]][y_allSenses_Sent_id[i][j][k]] = 1    
    
    with open(root_path + '/{}/{}/mask_wnDomain2sense.pkl'.format(dataset_name, file_name), "wb") as outfile:
        pickle.dump(mask_wnDomain2sense, outfile)
    
    print('mask_WnDomansToSenses is done ....')    
#     return mask_wnDomain2sense   



In [0]:
file_names = ['semcor', 'ALL', 'senseval2', 'senseval3', 'semeval2007', 'semeval2013', 'semeval2015']
dataset_name = 'semcor'

for file_name in file_names:
    mask_PosToSenses(root_path + '/{}/{}/y_allPos_Sent.pkl'.format(dataset_name, file_name), root_path + '/{}/{}/y_allSenses_Sent_id.pkl'.format(dataset_name, file_name), pos2id, SenseWords2id, dataset_name, file_name)

    mask_LexToSenses(root_path + '/{}/{}/y_allLex_Sent.pkl'.format(dataset_name, file_name), root_path + '/{}/{}/y_allSenses_Sent_id.pkl'.format(dataset_name, file_name), lex2id, SenseWords2id, dataset_name, file_name)

    mask_WnDomansToSenses(root_path + '/{}/{}/y_allWnDomain_Sent.pkl'.format(dataset_name, file_name), root_path + '/{}/{}/y_allSenses_Sent_id.pkl'.format(dataset_name, file_name), wnDomain2id, SenseWords2id, dataset_name, file_name)
    
    print('processing is ok for -----> ', file_name)
    

mask_PosToSenses is started ....
mask_PosToSenses is done ....
mask_LexToSenses is started ....
mask_LexToSenses is done ....
mask_WnDomansToSenses is started ....
mask_WnDomansToSenses is done ....
processing is ok for ----->  semcor
mask_PosToSenses is started ....
mask_PosToSenses is done ....
mask_LexToSenses is started ....
mask_LexToSenses is done ....
mask_WnDomansToSenses is started ....
mask_WnDomansToSenses is done ....
processing is ok for ----->  ALL
mask_PosToSenses is started ....
mask_PosToSenses is done ....
mask_LexToSenses is started ....
mask_LexToSenses is done ....
mask_WnDomansToSenses is started ....
mask_WnDomansToSenses is done ....
processing is ok for ----->  senseval2
mask_PosToSenses is started ....
mask_PosToSenses is done ....
mask_LexToSenses is started ....
mask_LexToSenses is done ....
mask_WnDomansToSenses is started ....
mask_WnDomansToSenses is done ....
processing is ok for ----->  senseval3
mask_PosToSenses is started ....
mask_PosToSenses is done